# 1: 切換 TensorFlow 版本及匯入所需套件

In [1]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [2]:
# Import some useful packages
import matplotlib.pyplot as plt
import numpy as np

# Layers for FNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

# Layers for CNN
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D

from tensorflow.keras.optimizers import SGD, Adam

# For data preprocessing
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

# 2: 下載並整理 Fashion MNIST 資料集

In [3]:
# Load Fashine MNIST
(X_train, y_train0), (X_test, y_test0) = mnist.load_data()

# Reshape size
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

# Normalize the range of featurs
X_train = X_train / X_train.max()
X_test = X_test / X_test.max()

# One-hot encoding
y_train = to_categorical(y_train0, 10)
y_test = to_categorical(y_test0, 10)

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

# 3.3 Functional API 的操作方式

In [5]:
#設定向量維度
f_1 = Dense(128, activation='sigmoid')
f_2 = Dense(128, activation='sigmoid')
f_3 = Dense(10, activation='softmax')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
print(f_1)

## 接著，定義層前後變數之間的關係；首先，第一個變數必定以 Input 函數來定義

In [7]:
x = Input(shape=(784,))

In [8]:
print(x)

Tensor("input_1:0", shape=(?, 784), dtype=float32)


In [9]:
h_1 = f_1(x)
h_2 = f_2(h_1)
y = f_3(h_2)

In [10]:
print(y)

Tensor("dense_2/Softmax:0", shape=(?, 10), dtype=float32)


## 在這裡，變數  h1,h2,y  是以張量 (tensor) 類別來表示，我們可以嘗試 print 看看。

In [11]:
print(h_1)
print(h_2)
print(y)

Tensor("dense/Sigmoid:0", shape=(?, 128), dtype=float32)
Tensor("dense_1/Sigmoid:0", shape=(?, 128), dtype=float32)
Tensor("dense_2/Softmax:0", shape=(?, 10), dtype=float32)


## 接著，透過 Model 將一個模型的輸入/輸出包裝起來，建立模型的過程就完成了！

In [12]:
model = Model(x, y)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## 讀入 MNIST 手寫辨識資料

In [19]:
model.compile(loss='mse', 
              optimizer=SGD(lr=0.05),  #learning rate設為 0.05
              metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, 
          batch_size=256, 
          epochs=5) #訓練次數 5 次

Epoch 1/5
60000/60000 [==============================] - 1s 23us/sample - loss: 0.0069 - acc: 0.9572
Epoch 2/5
60000/60000 [==============================] - 1s 20us/sample - loss: 0.0069 - acc: 0.9578
Epoch 3/5
60000/60000 [==============================] - 1s 20us/sample - loss: 0.0068 - acc: 0.9581
Epoch 4/5
60000/60000 [==============================] - 1s 23us/sample - loss: 0.0068 - acc: 0.9581
Epoch 5/5
60000/60000 [==============================] - 1s 20us/sample - loss: 0.0068 - acc: 0.9584


In [21]:
model.save_weights('MNIST_Model.h5')

In [22]:
score = model.evaluate(X_train, y_train, batch_size=10000)
print(f'Loss: {score[0]}')
print(f'準確率: {score[1]*100}')

60000/60000 [==============================] - 1s 10us/sample - loss: 0.0068 - acc: 0.9584
Loss: 0.006817058582479755
準確率: 95.8383321762085


# 4: 非線性堆疊模型

In [17]:
from tensorflow.keras.layers import concatenate, add

In [23]:
f_4 = Dense(300, activation='relu')
f_5 = Dense(300, activation='relu')
z = f_4(h_1)
y = f_5(z)

# new f_3
f_3 = Dense(10, activation='softmax')

In [24]:
f_3 = Dense(10, activation='softmax')

u = concatenate([h_2, z])
y = f_3(u)

In [25]:
print(u)
print(y)

Tensor("concatenate/concat:0", shape=(?, 428), dtype=float32)
Tensor("dense_8/Softmax:0", shape=(?, 10), dtype=float32)


In [29]:
model1 = Model(x, y)
model1.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          100480      input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          16512       dense[0][0]                      
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 300)          38700       dense[0][0]                      
____________________________________________________________________________________________

In [30]:
model1.compile(loss='mse', 
              optimizer=SGD(lr=0.05),  #learning rate設為 0.05
              metrics=['accuracy'])

In [31]:
model1.fit(X_train, y_train, 
          batch_size=256, 
          epochs=5) #訓練次數 5 次

Epoch 1/5
60000/60000 [==============================] - 2s 32us/sample - loss: 0.0294 - acc: 0.8586
Epoch 2/5
60000/60000 [==============================] - 2s 26us/sample - loss: 0.0250 - acc: 0.8837
Epoch 3/5
60000/60000 [==============================] - 2s 27us/sample - loss: 0.0209 - acc: 0.9339
Epoch 4/5
60000/60000 [==============================] - 2s 29us/sample - loss: 0.0179 - acc: 0.9419
Epoch 5/5
60000/60000 [==============================] - 2s 27us/sample - loss: 0.0159 - acc: 0.9445


In [33]:
score = model1.evaluate(X_train, y_train, batch_size=10000)
print(f'Loss: {score[0]}')
print(f'準確率: {score[1]*100}')

60000/60000 [==============================] - 1s 11us/sample - loss: 0.0151 - acc: 0.9454
Loss: 0.0151036293245852
準確率: 94.53833103179932
